In [1]:
import numpy as np
import torch

# open, high, low, close, volume, vwap
btc_2023 = np.fromfile('data/btc_2023.npy')
btc_2024 = np.fromfile('data/btc_2024.npy')

btc_2023 = torch.tensor(btc_2023.reshape(-1, 6)).T.float().cuda()
btc_2024 = torch.tensor(btc_2024.reshape(-1, 6)).T.float().cuda()


In [55]:
from model import WaveNet

model = WaveNet(nin=6, nout=2, chan=64, nlayer=[8,8,8]).cuda()


with torch.no_grad():
    y = model(btc_2023)

btc_2023.shape, y.shape
# y[:, -10:]

(torch.Size([6, 497979]), torch.Size([2, 497214]))

In [56]:
import numba

@numba.jit
def minimize(p_buy, p_sell):
    n = p_buy.size
    arcs = np.zeros((n, 2, 2), dtype=np.float32)
    arcs[:,0,1] = -p_buy
    arcs[:,1,0] = p_sell
    arcs[0,1,:] = -np.inf
    arcs[-1,:,1] = -np.inf

    for i in range(n-1, 0, -1):
        max0 = max(arcs[i,0,0], arcs[i,0,1])
        max1 = max(arcs[i,1,0], arcs[i,1,1])
        arcs[i,0,:] -= max0
        arcs[i,1,:] -= max1
        arcs[i-1,:,0] += max0
        arcs[i-1,:,1] += max1

    max0 = max(arcs[0,0,0], arcs[0,0,1])
    arcs[0,0,:] -= max0
    arcs[0,1,:] = 0
    arcs[-1,:,1] = 0
    
    return arcs, max0

In [63]:
fee = 0.01
p = btc_2023[None, ...]
p_sell = p[0, 2, :].cpu().numpy() * (1 - fee)
p_buy = p[0, 1, :].cpu().numpy() * (1 + fee)

arcs, profit = minimize(p_buy, p_sell)
print('max profit', profit)

lossw = arcs.transpose(1,2,0)[None,...]
lossw = torch.tensor(lossw).cuda()
print(lossw.shape)
print(lossw[..., 0])

lossw[0,0,0] *= 2
lossw[0,1,1] *= 2

lossw.abs().sum()


max profit 120907.984375
torch.Size([1, 2, 2, 497979])
tensor([[[  0.0000, -25.1562],
         [  0.0000,   0.0000]]], device='cuda:0')


tensor(2.9611e+08, device='cuda:0')

In [83]:
from tqdm import tqdm


softplus = torch.nn.functional.softplus

opt = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    betas=(0.9, 0.995),
    weight_decay=1e-5,
)
model.train()

chunk = 32768
loss_avg = 0
loss_w = 0

for _ in (prog := tqdm(range(1600))):
    opt.zero_grad()

    o = np.random.randint(0, p.shape[-1] - chunk)
    # o = np.random.randint(0, chunk)
    # o = 0

    y = model(p[..., o:o+chunk])[..., :-1]

    ypos = softplus(y)
    yneg = softplus(-y)
    
    o2 = o + chunk - y.shape[-1]
    loss = ( ypos * lossw[:, :, 1, o2:o+chunk]
           + yneg * lossw[:, :, 0, o2:o+chunk] )

    # loss = -y * lossw[..., -y.shape[-1]:]
    # loss = loss.sum() / lossw.abs().sum()
    loss = -loss.sum() / lossw[..., o2:o+chunk].abs().sum()

    loss_w = min(loss_w + 1, 100)
    loss_avg += (loss.item() - loss_avg) / loss_w
    
    prog.set_description(f'{loss_avg:.5f}')
    loss.backward()
    opt.step()

loss_avg

0.00500: 100%|██████████| 1600/1600 [00:26<00:00, 59.88it/s]


0.0050043745875863925

In [ ]:
import plotly.express as px

fee = 0.003
p_test = btc_2024[:, :]
p_sell = p_test[2, :].cpu().numpy() * (1 - fee)
p_buy = p_test[1, :].cpu().numpy() * (1 + fee)

model.eval()
with torch.no_grad():
    y = model(p_test)[..., :-1]
    
    y = y.sigmoid().cpu()

print(y.shape)
print(y.min(dim=-1).values)
print(y.max(dim=-1).values)

# px.line({'buy':y[0], 'sell':y[1]})


In [ ]:
@numba.jit
def sim(y, p_buy, p_sell):
    stock = 0
    cash = 0
    conf = 0.9
    trades = 0
    for i in range(y.shape[-1]):
        sell = (1 - y[1, i].item()) > conf
        buy = y[0, i].item() > conf

        if i == y.shape[-1] - 1:
            sell = 1
            buy = 0

        if stock==1 and sell:
            stock = 0
            cash += p_sell[i+1]
            trades += 1
        elif stock==0 and buy:
            stock = 1
            cash -= p_buy[i-1]
            trades += 1

    return cash, trades

sim(y.numpy(), p_buy[-y.shape[-1]:], p_sell[-y.shape[-1]:])

(30489.8046875, 82)